# Programming for Data Science and Artificial Intelligence

## 18. PyTorch

- [WEIDMAN] Ch7
- https://pytorch.org/tutorials/
- https://github.com/yunjey/pytorch-tutorial

Here we introduce PyTorch, an increasingly popular neural network framework based on **automatic differentiation**, which we introduced at the beginning of previous chapter.

As in the rest of the book, we’ll write our code in a way that maps to the mental models of how neural networks work, writing classes for Layers, Trainers, and so on. In doing so, we won’t be writing our code in line with common PyTorch practices, but I recommend you to watch the 60-min blitz to understand different features of pytorch (https://pytorch.org/tutorials/).  Pytorch is really cool and simple!  For example, there is a function <code>tensor.to(cuda)</code> so you can use the GPU to run any model which is much faster.  I highly recommend checking it out.

### Basics

1. Basic autograd example 1               
2. Basic autograd example 2               
3. Loading data from numpy                
4. Input pipline                          
5. Save and load model 

#### 1. Basic autograd example 1

In [31]:
import torch 

# Create tensors.
# only tensors of floating point dtype can get gradient
x = torch.tensor(1., requires_grad=True)
w = torch.tensor(2., requires_grad=True)
b = torch.tensor(3., requires_grad=True)

# Build a computational graph.
y = w * x + b    # y = 2 * x + 3

# Compute gradients
# Pytorch tensor can automatically compute the derivative
# of the parameters in respect to loss
y.backward()

# Print out the gradients.
print("Gradient of x: ", x.grad)    # x.grad = 2 
print("Gradient of w: ", w.grad)    # w.grad = 1 
print("Gradient of b: ", b.grad)    # b.grad = 1 

Gradient of x:  tensor(2.)
Gradient of w:  tensor(1.)
Gradient of b:  tensor(1.)


#### 2. Basic autograd example 2

In [32]:
import torch.nn as nn

# Create tensors of shape (10, 3) and (10, 2).
x = torch.randn(10, 3)
y = torch.randn(10, 2)

# Build a fully connected layer.
linear = nn.Linear(3, 2)
print ('w: ', linear.weight)
print ('b: ', linear.bias)

# Build loss function and optimizer.
criterion = nn.MSELoss()
optimizer = torch.optim.SGD(linear.parameters(), lr=0.01)

# Forward pass.
pred = linear(x)

# Compute loss.
loss = criterion(pred, y)
print('loss: ', loss.item())

# Backward pass.
loss.backward()

# Print out the gradients.
print ('dL/dw: ', linear.weight.grad) 
print ('dL/db: ', linear.bias.grad)

# 1-step gradient descent.
optimizer.step()

# You can also perform gradient descent at the low level.
# linear.weight.data.sub_(0.01 * linear.weight.grad.data)
# linear.bias.data.sub_(0.01 * linear.bias.grad.data)

# Print out the loss after 1-step gradient descent.
pred = linear(x)
loss = criterion(pred, y)
print('loss after 1 step optimization: ', loss.item())

w:  Parameter containing:
tensor([[ 0.5373,  0.3946, -0.2021],
        [ 0.0053,  0.1415, -0.4333]], requires_grad=True)
b:  Parameter containing:
tensor([ 0.1422, -0.0619], requires_grad=True)
loss:  1.4837201833724976
dL/dw:  tensor([[ 0.2383,  0.2634, -0.8096],
        [ 0.1286,  0.2615, -0.3749]])
dL/db:  tensor([-0.3221,  0.0213])
loss after 1 step optimization:  1.4726998805999756


#### 3. Loading data from numpy

In [33]:
import numpy as np

# Create a numpy array.
x = np.array([[1, 2], [3, 4]])

# Convert the numpy array to a torch tensor.
y = torch.from_numpy(x)

# Convert the torch tensor to a numpy array.
z = y.numpy()

#### 4. Input pipeline

In [34]:
import torchvision
import torchvision.transforms as transforms

# Download and construct CIFAR-10 dataset.
train_dataset = torchvision.datasets.CIFAR10(root='data',
                                             train=True, 
                                             transform=transforms.ToTensor(),
                                             download=True)

Files already downloaded and verified


In [35]:
# Fetch one data pair (read data from disk).
image, label = train_dataset[0]
print (image.size())
print (label)

torch.Size([3, 32, 32])
6


In [36]:
# Retrieve batch of data
# Data loader (this provides queues and threads in a very simple way).
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=64, 
                                           shuffle=True)
# When iteration starts, queue and thread start to load data from files.
data_iter = iter(train_loader)

# Mini-batch images and labels.
images, labels = data_iter.next()

print(images.size())
print(labels.size())

torch.Size([64, 3, 32, 32])
torch.Size([64])


In [37]:
# Actual usage of the data loader is as below.
for images, labels in train_loader:
    # Training code should be written here.
    pass

#### 5. Save and load the model

In [41]:
# Save and load the entire model.
torch.save(resnet, 'models/model.ckpt')
model = torch.load('models/model.ckpt')

# Save and load only the model parameters (recommended).
torch.save(resnet.state_dict(), 'models/params.ckpt')
resnet.load_state_dict(torch.load('models/params.ckpt'))

<All keys matched successfully>

### Linear Regression

Consider this data:

<img src = "figures/japan.png">

n a linear regression model, each target variable is estimated to be a weighted sum of the input variables, offset by some constant, known as a bias :

$$yield_{apple}  = w_{11} * temp + w_{12} * rainfall + w_{13} * humidity + b_{1}$$

$$yield_{orange} = w_{21} * temp + w_{22} * rainfall + w_{23} * humidity + b_{2}$$

Visually, it means that the yield of apples is a linear or planar function of temperature, rainfall and humidity:

<img src = "figures/japan2.png">

The learning part of linear regression is to figure out a set of weights <code>w11, w12,... w23, b1 \& b2</code> by looking at the training data, to make accurate predictions for new data (i.e. to predict the yields for apples and oranges in a new region using the average temperature, rainfall and humidity). This is done by adjusting the weights slightly many times to make better predictions, using an optimization technique called gradient descent

#### Training data

In [49]:
# Input (temp, rainfall, humidity)
inputs = np.array([[73, 67, 43], [91, 88, 64], [87, 134, 58], 
                   [102, 43, 37], [69, 96, 70], [73, 67, 43], 
                   [91, 88, 64], [87, 134, 58], [102, 43, 37], 
                   [69, 96, 70], [73, 67, 43], [91, 88, 64], 
                   [87, 134, 58], [102, 43, 37], [69, 96, 70]], 
                  dtype='float32')

# Targets (apples, oranges)
targets = np.array([[56, 70], [81, 101], [119, 133], 
                    [22, 37], [103, 119], [56, 70], 
                    [81, 101], [119, 133], [22, 37], 
                    [103, 119], [56, 70], [81, 101], 
                    [119, 133], [22, 37], [103, 119]], 
                   dtype='float32')

inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)
print(inputs.size())
print(targets.size())

torch.Size([15, 3])
torch.Size([15, 2])


#### Dataset and DataLoader

We'll create a TensorDataset, which allows access to rows from inputs and targets as tuples, and provides standard APIs for working with many different types of datasets in PyTorch.

In [50]:
from torch.utils.data import TensorDataset

In [52]:
# Define dataset
train_ds = TensorDataset(inputs, targets)
train_ds[0:3]

(tensor([[ 73.,  67.,  43.],
         [ 91.,  88.,  64.],
         [ 87., 134.,  58.]]),
 tensor([[ 56.,  70.],
         [ 81., 101.],
         [119., 133.]]))

The TensorDataset allows us to access a small section of the training data using the array indexing notation (<code>[0:3]</code> in the above code). It returns a tuple (or pair), in which the first element contains the input variables for the selected rows, and the second contains the targets.

We'll also create a <code>DataLoader</code>, which can split the data into batches of a predefined size while training. It also provides other utilities like shuffling and random sampling of the data.

In [53]:
from torch.utils.data import DataLoader

In [54]:
# Define data loader
batch_size = 5
train_dl = DataLoader(train_ds, batch_size, shuffle=True)

The data loader is typically used in a for-in loop. Let's look at an example

In [55]:
for xb, yb in train_dl:
    print(xb)
    print(yb)
    break

tensor([[ 69.,  96.,  70.],
        [102.,  43.,  37.],
        [102.,  43.,  37.],
        [ 73.,  67.,  43.],
        [ 87., 134.,  58.]])
tensor([[103., 119.],
        [ 22.,  37.],
        [ 22.,  37.],
        [ 56.,  70.],
        [119., 133.]])


In each iteration, the data loader returns one batch of data, with the given batch size. If shuffle is set to True, it shuffles the training data before creating batches. Shuffling helps randomize the input to the optimization algorithm, which can lead to faster reduction in the loss.

#### nn.Linear

Instead of initializing the weights & biases manually, we can define the model using the nn.Linear class from PyTorch, which does it automatically.

In [59]:
# Define model
model = nn.Linear(3, 2)  #nn.Linear assume this shape (in_features, out_features)
print(model.weight)
print(model.weight.size()) # (out_features, in_features)
print(model.bias)
print(model.bias.size()) #(out_features)

Parameter containing:
tensor([[ 0.0305,  0.4500,  0.3552],
        [-0.0460, -0.3471,  0.3826]], requires_grad=True)
torch.Size([2, 3])
Parameter containing:
tensor([ 0.4901, -0.4545], requires_grad=True)
torch.Size([2])


In fact, our model is simply a function that performs a matrix multiplication of the <code>inputs</code> and the weights <code>w</code> and adds the bias <code>b</code> (for each observation)

<img src = "figures/dot.png" width="400">

PyTorch models also have a helpful <code>.parameters</code> method, which returns a list containing all the weights and bias matrices present in the model. For our linear regression model, we have one weight matrix and one bias matrix.

In [62]:
# Parameters
list(model.parameters())  #model.param returns a generator

[Parameter containing:
 tensor([[ 0.0305,  0.4500,  0.3552],
         [-0.0460, -0.3471,  0.3826]], requires_grad=True),
 Parameter containing:
 tensor([ 0.4901, -0.4545], requires_grad=True)]

We can use the <code>model(tensor)</code> API to perform a forward-pass that generate predictions

In [63]:
# Generate predictions
preds = model(inputs)
preds

tensor([[ 48.1414, -10.6209],
        [ 65.6002, -10.7049],
        [ 84.0460, -28.7849],
        [ 36.0949,  -5.9195],
        [ 70.6605, -10.1742],
        [ 48.1414, -10.6209],
        [ 65.6002, -10.7049],
        [ 84.0460, -28.7849],
        [ 36.0949,  -5.9195],
        [ 70.6605, -10.1742],
        [ 48.1414, -10.6209],
        [ 65.6002, -10.7049],
        [ 84.0460, -28.7849],
        [ 36.0949,  -5.9195],
        [ 70.6605, -10.1742]], grad_fn=<AddmmBackward>)

#### Loss function

The <code>nn</code> module contains a lot of useful loss function like this:

In [65]:
mean_squared_error_loss = nn.MSELoss()
softmax_cross_entropy_loss = nn.CrossEntropyLoss()

In [67]:
mse = mean_squared_error_loss(preds, targets)
print(mse)

tensor(6644.5342, grad_fn=<MseLossBackward>)


#### Optimizer

*Learning rate and momentum**

Instead of manually manipulating the model's weights & biases using gradients, we can use <code>torch.optim</code> API.  We can use <code>optim.SGD</code> to perform stochastic gradient descent where samples are selected in batches (often with random shuffling) instead of as a single group.

In [68]:
# Define optimizer
opt = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

Let's talk a bit about other techniques we have used before and how they look like in PyTorch.  Since this is simple linear regression problem, we only use learning rate and momentum, but it's worth to talk it here and perhaps we can see the code in the neural network section

*Weight initialization*

We don’t need to worry about weight initialization at all: the weights in most PyTorch operations involving parameters, including nn.Linear, are automatically scaled based on the size of the layer.

*Dropout*

Dropout is similarly easy. Just as PyTorch has a built-in Module <code>nn.Linear(n_in, n_out)</code> that computes the operations of a Dense layer from before, the Module nn.Dropout(dropout_prob) implements the Dropout operation, with the caveat that the probability passed in is by default the probability of dropping a given neuron.

*Learning rate decay*

PyTorch has an <code>lr_scheduler</code> class that can be used to decay the learning rate over the epochs. The key import you need to get started is <code>from torch.optim import lr_scheduler</code>   We shall use it in the latter section